![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Module 3: Data Engineering for Context

## From RAG Basics to Practical Context Engineering

In Module 2, you built a working RAG system and saw why context quality matters. Now you'll learn to engineer context with professional-level rigor.

**What makes context "good"?**

This module teaches you that **context engineering is real engineering** - it requires the same rigor, analysis, and deliberate decision-making as any other engineering discipline. Context isn't just "data you feed to an LLM" - it requires thoughtful preparation, quality assessment, and optimization.

## What You'll Learn

**The Engineering Mindset:**
- Why context quality matters (concrete impact on accuracy, relevance, cost)
- The transformation workflow: Raw Data → Engineered Context → Quality Responses
- Contrasts between naive and engineered approaches

**Data Engineering for Context:**
- Systematic transformation: Extract → Clean → Transform → Optimize → Store
- Engineering decisions based on YOUR domain requirements
- When to use different approaches (RAG, Structured Views, Hybrid)

**Introduction to Chunking:**
- When does your data need chunking? (Critical first question)
- Different chunking strategies and their trade-offs
- How to choose based on YOUR data characteristics

**Context Preparation Pipelines:**
- Three pipeline architectures (Request-Time, Batch, Event-Driven)
- How to choose based on YOUR constraints
- Building reusable context preparation workflows

**⏱️ Estimated Time:** 90-105 minutes

---

## Prerequisites

- Completed Module 2: RAG Fundamentals and Implementation
- Redis 8 running locally
- OpenAI API key set
- Understanding of RAG basics and vector embeddings

---

## Part 1: Context is Data - and Data Requires Engineering

### The Naive Approach (What NOT to Do)

Let's start by seeing what happens when you treat context as "just data" without engineering discipline.

**Scenario:** A student asks "What machine learning courses are available?"

Let's see what happens with a naive approach:

### Setup

In [1]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv

# Handle both running from workshop/ directory and from project root
if Path.cwd().name == "workshop":
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables from project root
env_path = project_root / ".env"
load_dotenv(dotenv_path=env_path)

# Verify required environment variables
required_vars = ["OPENAI_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"""⚠️  Missing required environment variables: {', '.join(missing_vars)}

Please create a .env file with:
OPENAI_API_KEY=your_openai_api_key
REDIS_URL=redis://localhost:6379
""")
    sys.exit(1)

REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print("✅ Environment variables loaded")

✅ Environment variables loaded


In [2]:
import asyncio
import json
from typing import Any, Dict, List

import redis
import tiktoken
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Import hierarchical components (from Module 2)
from redis_context_course.hierarchical_manager import HierarchicalCourseManager
from redis_context_course.hierarchical_context import HierarchicalContextAssembler
from redis_context_course import CourseManager, redis_config

# Initialize
course_manager = CourseManager()
hierarchical_manager = HierarchicalCourseManager(redis_client=redis.from_url(REDIS_URL, decode_responses=True))
context_assembler = HierarchicalContextAssembler()
redis_client = redis.from_url(REDIS_URL, decode_responses=True)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Token counter
encoding = tiktoken.encoding_for_model("gpt-4o")


def count_tokens(text: str) -> int:
    return len(encoding.encode(text))


print("✅ Dependencies loaded")

19:13:57 redisvl.index.index INFO   Index already exists, not overwriting.


✅ Dependencies loaded


### Naive Approach: Dump Everything

The simplest approach is to include all course data in every request:

In [3]:
# Naive Approach: Get all courses and dump as JSON
all_courses = await course_manager.get_all_courses()

# Convert to raw JSON (what many developers do first)
raw_context = json.dumps(
    [
        {
            "id": c.id,
            "course_code": c.course_code,
            "title": c.title,
            "description": c.description,
            "department": c.department,
            "credits": c.credits,
            "difficulty_level": c.difficulty_level.value,
            "format": c.format.value,
            "instructor": c.instructor,
            "prerequisites": (
                [p.course_code for p in c.prerequisites] if c.prerequisites else []
            ),
            "created_at": str(c.created_at) if hasattr(c, "created_at") else None,
            "updated_at": str(c.updated_at) if hasattr(c, "updated_at") else None,
        }
        for c in all_courses[:10]  # Just first 10 for demo
    ],
    indent=2,
)

token_count = count_tokens(raw_context)

print(f"""📊 Naive Approach Results:
   Courses included: {len(all_courses[:10])}
   Token count: {token_count:,}
   Estimated cost per request: ${(token_count / 1_000_000) * 2.50:.4f}

   For 100 courses, this would be ~{token_count * 10:,} tokens!
""")

# Show a sample
print("\n📄 Sample of raw JSON context:")
print(raw_context[:500] + "...")

19:13:58 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


📊 Naive Approach Results:
   Courses included: 10
   Token count: 2,053
   Estimated cost per request: $0.0051

   For 100 courses, this would be ~20,530 tokens!


📄 Sample of raw JSON context:
[
  {
    "id": "course_catalog:01KB16WAF4TQ2W7EBDBRNPW201",
    "course_code": "CS008",
    "title": "Natural Language Processing",
    "description": "Processing and understanding human language with machine learning. This course covers text processing, language models, sentiment analysis, and modern NLP techniques. Students will work with transformers, BERT, and GPT models. Projects include building chatbots, text classifiers, and language generation systems.",
    "department": "Computer Sci...


In [4]:
[course.title for course in all_courses]

['Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Natural Language Processing',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Linear Algebra for Machine Learning',
 'Computer Vision',
 'Computer Vision',
 'Computer Vision',
 'Computer Vision',
 'Computer Vision',
 'Computer Vision',
 'Computer Vision',
 'Computer Vision',
 'Deep Learning and Neural Networks',
 'Deep Learning and Neural Networks',
 'De

### Test the Naive Approach

In [5]:
# Test with a real query
query = "What machine learning courses are available?"

messages = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

Available Courses:
{raw_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]

response = llm.invoke(messages)

print(f"""🤖 Query: "{query}"

Response:
{response.content}
""")

19:14:02 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Query: "What machine learning courses are available?"

Response:
The available course related to machine learning is "Natural Language Processing." This course focuses on processing and understanding human language with machine learning techniques. It covers text processing, language models, sentiment analysis, and modern NLP techniques, including working with transformers, BERT, and GPT models. Students will engage in projects such as building chatbots, text classifiers, and language generation systems.

Here are the different formats and instructors for this course:

1. **Hybrid Format**
   - Instructor: Cindy Sandoval (Course Code: CS008)
   - Instructor: Joshua Smith (Course Code: CS037)
   - Instructor: Frank Walker (Course Code: CS010)
   - Instructor: Julie Bell (Course Code: CS042)

2. **Online Format**
   - Instructor: Nichole Moss (Course Code: CS018)
   - Instructor: Phillip Perez (Course Code: CS048)
   - Instructor: Shannon Reed (Course Code: CS050)

3. **In-Person Forma

### Problems with the Naive Approach

As discussed in Module 2, this approach has several problems:

1. **Excessive Token Usage**
   - 10 courses = ~1,703 tokens
   - 100 courses would be ~17,030 tokens


2. **Raw JSON is Inefficient**
   - Includes internal fields (IDs, timestamps, created_at, updated_at)
   - Verbose formatting (indentation, field names repeated)


3. **No Filtering**
   - Student asked about ML, but got all courses, even irrelevant ones
   - **Dilutes relevant information with noise**


4. **Poor Response Quality**
   - Generic responses ("We have many courses...")
   - May miss the most relevant courses
   - Can't provide personalized recommendations


5. **Not Scalable**
   - What if you have 1,000 courses? 10,000?
   - What if courses change daily?
   - Requires code changes to update

**Therefore, the goal is not only to give the LLM "all the data" - it's to *give it the useful data.***

---

## The Engineering Mindset

Context is data that flows through a pipeline. Like any data engineering problem, it requires:

### 1. Requirements Analysis
- What is the intended use case?
- What queries will users ask?
- What information do they need?
- What constraints exist (token budget, latency, cost)?

### 2. Data Transformation
- Raw data → Cleaned data → Structured data → LLM-optimized context

### 3. Quality Metrics
- How do we measure if context is "good"?
- Relevance, completeness, efficiency, accuracy

### 4. Testing and Iteration
- Test with real queries
- Measure quality metrics
- Iterate based on results

**The Engineering Question:** "How do we transform raw course data into high-quality context that produces accurate, relevant, efficient responses?"

### Three Engineering Approaches

Let's compare three approaches with concrete examples:

| Approach | Description | Token Usage | Response Quality | Maintenance | Verdict |
|----------|-------------|-------------|------------------|-------------|---------|
| **Naive** | Include all raw data | 50K tokens | Poor (generic) | Easy | ❌ Not practical |
| **RAG** | Semantic search for relevant courses | 3K tokens | Good (relevant) | Moderate | ✅ Good for most cases |
| **Structured Views** | Pre-compute LLM-optimized summaries | 2K tokens | Excellent (overview + details) | Higher | ✅ Best for real-world use |
| **Hybrid** | Structured view + RAG | 5K tokens | Excellent (best of both) | Higher | ✅ Best for real-world use |

Let's implement each approach and compare them.

---

## Part 2: Data Engineering Workflow - From Raw to Optimized

### The Data Engineering Pipeline

Context preparation follows a systematic workflow:

```
Raw Data (Database/API)
    ↓
1. EXTRACT: Pull relevant data
    ↓
2. CLEAN: Remove noise, fix inconsistencies
    ↓
3. TRANSFORM: Structure for LLM consumption
    ↓
4. OPTIMIZE: Reduce tokens, improve relevance
    ↓
5. STORE: Index for efficient retrieval
    ↓
Optimized Context (Ready for LLM)
```

Let's implement each step:

### Step 1: Extract (Raw Data)

First, let's look at what raw course data looks like:

In [6]:
# Get a sample course
sample_course = all_courses[0]

# Show raw database record
raw_record = {
    "id": sample_course.id,
    "course_code": sample_course.course_code,
    "title": sample_course.title,
    "description": sample_course.description,
    "department": sample_course.department,
    "credits": sample_course.credits,
    "difficulty_level": sample_course.difficulty_level.value,
    "format": sample_course.format.value,
    "instructor": sample_course.instructor,
    "prerequisites": (
        [p.course_code for p in sample_course.prerequisites]
        if sample_course.prerequisites
        else []
    ),
    "created_at": (
        str(sample_course.created_at)
        if hasattr(sample_course, "created_at")
        else "2024-01-15T08:30:00Z"
    ),
    "updated_at": (
        str(sample_course.updated_at)
        if hasattr(sample_course, "updated_at")
        else "2024-09-01T14:22:00Z"
    ),
}

raw_json = json.dumps(raw_record, indent=2)
raw_tokens = count_tokens(raw_json)

print("📄 Step 1: Raw Database Record")
print("=" * 80)
print(raw_json)
print("=" * 80)
print(f"\n📊 Token count: {raw_tokens}")

📄 Step 1: Raw Database Record
{
  "id": "course_catalog:01KB16WAF4TQ2W7EBDBRNPW201",
  "course_code": "CS008",
  "title": "Natural Language Processing",
  "description": "Processing and understanding human language with machine learning. This course covers text processing, language models, sentiment analysis, and modern NLP techniques. Students will work with transformers, BERT, and GPT models. Projects include building chatbots, text classifiers, and language generation systems.",
  "department": "Computer Science",
  "credits": 3,
  "difficulty_level": "advanced",
  "format": "hybrid",
  "instructor": "Cindy Sandoval",
  "prerequisites": [
    "CS001"
  ],
  "created_at": "2025-12-04 19:13:58.502499",
  "updated_at": "2025-12-04 19:13:58.502505"
}

📊 Token count: 204


Issues with above:
 - Internal fields (IDs, timestamps) waste tokens
 - Verbose JSON formatting
 - Prerequisites are codes, not human-readable
 - No structure for LLM consumption

### Step 2: Clean (Remove Noise)

Remove fields that don't help the LLM answer user queries:

In [7]:
# Step 2: Clean - Remove internal fields
cleaned_record = {
    "course_code": sample_course.course_code,
    "title": sample_course.title,
    "description": sample_course.description,
    "department": sample_course.department,
    "credits": sample_course.credits,
    "difficulty_level": sample_course.difficulty_level.value,
    "format": sample_course.format.value,
    "instructor": sample_course.instructor,
    "prerequisites": (
        [p.course_code for p in sample_course.prerequisites]
        if sample_course.prerequisites
        else []
    ),
}

cleaned_json = json.dumps(cleaned_record, indent=2)
cleaned_tokens = count_tokens(cleaned_json)

print("📄 Step 2: Cleaned Record")
print("=" * 80)
print(cleaned_json)
print("=" * 80)
print(f"\n📊 Token count: {cleaned_tokens} (saved {raw_tokens - cleaned_tokens} tokens, {((raw_tokens - cleaned_tokens) / raw_tokens * 100):.1f}% reduction)")

📄 Step 2: Cleaned Record
{
  "course_code": "CS008",
  "title": "Natural Language Processing",
  "description": "Processing and understanding human language with machine learning. This course covers text processing, language models, sentiment analysis, and modern NLP techniques. Students will work with transformers, BERT, and GPT models. Projects include building chatbots, text classifiers, and language generation systems.",
  "department": "Computer Science",
  "credits": 3,
  "difficulty_level": "advanced",
  "format": "hybrid",
  "instructor": "Cindy Sandoval",
  "prerequisites": [
    "CS001"
  ]
}

📊 Token count: 134 (saved 70 tokens, 34.3% reduction)



Improvements:
 - Removed id, created_at, updated_at
 - Still has all information needed to answer queries

Still has minor problems:
 - JSON formatting is verbose (this is a *minor* issue as LLMs can handle it; however)
 - Prerequisites are still codes


### Step 3: Transform (Structure for LLM)

Convert to a format optimized for LLM consumption:

In [8]:
# Step 3: Transform - Convert to LLM-friendly format


def transform_course_to_text(course) -> str:
    """Transform course object to LLM-optimized text format."""

    # Build prerequisites text
    prereq_text = ""
    if course.prerequisites:
        prereq_codes = [p.course_code for p in course.prerequisites]
        prereq_text = f"\nPrerequisites: {', '.join(prereq_codes)}"

    # Build course text
    course_text = f"""{course.course_code}: {course.title}
Department: {course.department}\nCredits: {course.credits}\nLevel: {course.difficulty_level.value}\nFormat: {course.format.value}
Instructor: {course.instructor}{prereq_text}
Description: {course.description}
    """

    return course_text


transformed_text = transform_course_to_text(sample_course)
transformed_tokens = count_tokens(transformed_text)

print("📄 Step 3: Transformed to LLM-Friendly Format")
print("=" * 80)
print(transformed_text)
print("=" * 80)
print(f"\n📊 Token count: {transformed_tokens} (saved {cleaned_tokens - transformed_tokens} tokens, {((cleaned_tokens - transformed_tokens) / cleaned_tokens * 100):.1f}% reduction)")

📄 Step 3: Transformed to LLM-Friendly Format
CS008: Natural Language Processing
Department: Computer Science
Credits: 3
Level: advanced
Format: hybrid
Instructor: Cindy Sandoval
Prerequisites: CS001
Description: Processing and understanding human language with machine learning. This course covers text processing, language models, sentiment analysis, and modern NLP techniques. Students will work with transformers, BERT, and GPT models. Projects include building chatbots, text classifiers, and language generation systems.
    

📊 Token count: 93 (saved 41 tokens, 30.6% reduction)



✅ Improvements:
 - Natural text format with the correct metadata
 - Clear structure with labels
 - No JSON overhead (brackets, quotes, commas)

**Note:** In case the description is too long, we can apply compression techniques such as summarization to keep the description within a desired token limit. Module 4 will cover compression in more detail.


### Step 4: Optimize (Further Reduce Tokens)

For even more efficiency, we can create a summarized version:

In [9]:
# Step 4: Optimize - Create ultra-compact version

def optimize_course_text(course) -> str:
    """Create ultra-compact course description."""
    prereqs = (
        f" (Prereq: {', '.join([p.course_code for p in course.prerequisites])})"
        if course.prerequisites
        else ""
    )
    return (
        f"{course.course_code}: {course.title} - {course.description[:100]}...{prereqs}"
    )


optimized_text = optimize_course_text(sample_course)
optimized_tokens = count_tokens(optimized_text)

print("📄 Step 4: Optimized (Ultra-Compact)")
print("=" * 80)
print(optimized_text)
print("=" * 80)
print(f"\n📊 Token count: {optimized_tokens} (saved {transformed_tokens - optimized_tokens} tokens, {((transformed_tokens - optimized_tokens) / transformed_tokens * 100):.1f}% reduction)")

📄 Step 4: Optimized (Ultra-Compact)
CS008: Natural Language Processing - Processing and understanding human language with machine learning. This course covers text processin... (Prereq: CS001)

📊 Token count: 30 (saved 63 tokens, 67.7% reduction)


Improvements:
   - Truncated description to 100 chars
   - Removed metadata (instructor, format, credits)

Trade-off:
   - Lost some detail (may need for specific queries)
   - Best for overview/catalog views

**Note:** This is just an example of what you can do to be more efficient. This is where you have to be creative and engineer based on the use case and requirements.

### Step 5: Store (Choose Storage Strategy)

Now we need to decide HOW to store this engineered context:

**Option 1: Vector Database (RAG)**
- Store transformed text with embeddings
- Retrieve relevant courses at query time
- Good for: Large datasets, specific queries

**Option 2: Pre-Computed Views**
- Create structured summaries ahead of time
- Store in Redis as cached views
- Good for: Common queries, overview information

**Option 3: Hybrid**
- Combine both approaches
- Pre-compute catalog view + RAG for details
- Good for: Real-world systems

Let's implement all three and compare.

### Summary: The Transformation Pipeline

Let's see the complete transformation:

In [10]:
print("=" * 80)
print("EXAMPLE PIPELINE SUMMARY")
print("=" * 80)

print(f"""
Step 1: Raw Database Record
   Token count: {raw_tokens}
   Format: JSON with all fields

Step 2: Cleaned Record
   Token count: {cleaned_tokens} ({((raw_tokens - cleaned_tokens) / raw_tokens * 100):.1f}% reduction)
   Removed: Internal fields (IDs, timestamps)

Step 3: Transformed to LLM Format
   Token count: {transformed_tokens} ({((cleaned_tokens - transformed_tokens) / cleaned_tokens * 100):.1f}% reduction from Step 2)
   Format: Natural text, structured

Step 4: Optimized (Ultra-Compact)
   Token count: {optimized_tokens} ({((transformed_tokens - optimized_tokens) / transformed_tokens * 100):.1f}% reduction from Step 3)
   Format: Single line, truncated

TOTAL REDUCTION: {raw_tokens} → {optimized_tokens} tokens ({((raw_tokens - optimized_tokens) / raw_tokens * 100):.1f}% reduction)
""")

print("=" * 80)
print("\n🎯 Key Insight:")
print("   Through systematic engineering, we reduced token usage by ~70%")
print("   while IMPROVING readability for the LLM!")
print("=" * 80)

EXAMPLE PIPELINE SUMMARY

Step 1: Raw Database Record
   Token count: 204
   Format: JSON with all fields

Step 2: Cleaned Record
   Token count: 134 (34.3% reduction)
   Removed: Internal fields (IDs, timestamps)

Step 3: Transformed to LLM Format
   Token count: 93 (30.6% reduction from Step 2)
   Format: Natural text, structured

Step 4: Optimized (Ultra-Compact)
   Token count: 30 (67.7% reduction from Step 3)
   Format: Single line, truncated

TOTAL REDUCTION: 204 → 30 tokens (85.3% reduction)


🎯 Key Insight:
   Through systematic engineering, we reduced token usage by ~70%
   while IMPROVING readability for the LLM!


The key insight states that we reduced token usage.

However, it should be noted that reduction is not the goal. The goal is to optimize the content and provide the most relevant information to the LLM.

---

## Part 3: Engineering Decision - When to Use Each Approach

Now let's implement the three approaches and compare them with real queries.

### Approach 1: RAG (Semantic Search)

Retrieve only relevant courses using vector search:

In [11]:
# Approach 1: RAG with Hierarchical Search
# Use the hierarchical manager from Module 2

async def rag_approach(query: str, summary_limit: int = 5, detail_limit: int = 3) -> str:
    """Retrieve relevant courses using hierarchical semantic search."""

    # Use hierarchical search (summaries + details)
    summaries, details = await hierarchical_manager.hierarchical_search(
        query=query,
        summary_limit=summary_limit,
        detail_limit=detail_limit
    )

    # Use context assembler for LLM-friendly format
    context = context_assembler.assemble_hierarchical_context(
        summaries=summaries,
        details=details,
        query=query
    )
    return context


# Test RAG approach
query = "What machine learning courses are available?"
rag_context = await rag_approach(query)
rag_tokens = count_tokens(rag_context)

print(f"""📊 RAG Approach Results:
   Query: "{query}"
   Token count: {rag_tokens:,}

📄 Context Preview:
{rag_context[:500]}...
""")

19:14:02 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What machine learning courses are available?' (summaries=5, details=3)


19:14:02 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:14:02 redisvl.index.index INFO   Index already exists, not overwriting.


19:14:02 redis_context_course.hierarchical_manager INFO   Created summary index: course_summaries


19:14:02 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: What machine learning courses are available?


19:14:02 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:14:02 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


📊 RAG Approach Results:
   Query: "What machine learning courses are available?"
   Token count: 25

📄 Context Preview:
# Course Search Results for: What machine learning courses are available?

## Overview of All Matches

Found 0 relevant courses:
...



### Approach 2: Structured Views (Pre-Computed Summaries)

Create a pre-computed catalog view that's optimized for LLM consumption:

In [12]:
# Approach 2: Structured Views
# Pre-compute a catalog summary organized by department


async def create_catalog_view() -> str:
    """Create a pre-computed catalog view organized by department."""

    # Group courses by department
    by_department = {}
    for course in all_courses:
        dept = course.department
        if dept not in by_department:
            by_department[dept] = []
        by_department[dept].append(course)

    # Build catalog view
    catalog_sections = []

    for dept_name in sorted(by_department.keys()):
        courses = by_department[dept_name]

        # Create department section
        dept_section = f"\n## {dept_name} ({len(courses)} courses)\n"

        # Add course summaries (optimized format)
        course_summaries = []
        for course in courses[:10]:  # Limit for demo
            summary = f"- {course.course_code}: {course.title} ({course.difficulty_level.value})"
            course_summaries.append(summary)

        dept_section += "\n".join(course_summaries)
        catalog_sections.append(dept_section)

    catalog_view = "# Redis University Course Catalog\n" + "\n".join(catalog_sections)
    return catalog_view


# Create and cache the view
catalog_view = await create_catalog_view()
catalog_tokens = count_tokens(catalog_view)

# Store in Redis for reuse
redis_client.set("course_catalog_view", catalog_view)

print(f"""📊 Structured View Approach Results:
   Total courses: {len(all_courses)}
   Token count: {catalog_tokens:,}
   Cached in Redis: ✅

📄 Catalog Preview:
{catalog_view[:600]}...
""")

📊 Structured View Approach Results:
   Total courses: 50
   Token count: 259
   Cached in Redis: ✅

📄 Catalog Preview:
# Redis University Course Catalog

## Computer Science (39 courses)
- CS008: Natural Language Processing (advanced)
- CS018: Natural Language Processing (advanced)
- CS020: Natural Language Processing (advanced)
- CS027: Natural Language Processing (advanced)
- CS034: Natural Language Processing (advanced)
- CS037: Natural Language Processing (advanced)
- CS048: Natural Language Processing (advanced)
- CS050: Natural Language Processing (advanced)
- CS010: Natural Language Processing (advanced)
- CS042: Natural Language Processing (advanced)

## Mathematics (11 courses)
- MATH001: Linear Algeb...



### Approach 3: Hybrid (Best of Both Worlds)

Combine structured view (overview) + RAG (specific details):

In [13]:
# Approach 3: Hybrid


async def hybrid_approach(query: str) -> str:
    """Combine catalog overview with RAG for specific details."""

    # Part 1: Get catalog overview (from cache)
    catalog_overview = redis_client.get("course_catalog_view")

    # Part 2: Get specific course details via hierarchical RAG
    specific_courses = await rag_approach(query, summary_limit=3, detail_limit=2)

    # Combine
    hybrid_context = f"""# Course Catalog Overview
{catalog_overview}

---

# Detailed Information for Your Query
{specific_courses}
"""

    return hybrid_context


# Test hybrid approach
hybrid_context = await hybrid_approach(query)
hybrid_tokens = count_tokens(hybrid_context)

print(f"""📊 Hybrid Approach Results:
   Query: "{query}"
   Token count: {hybrid_tokens:,}

   Components:
   - Catalog overview: {catalog_tokens:,} tokens
   - Specific details (RAG): {rag_tokens:,} tokens

📄 Context Structure:
   1. Full catalog overview (all departments)
   2. Detailed info for most relevant courses
""")

19:14:02 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What machine learning courses are available?' (summaries=3, details=2)


19:14:03 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:14:03 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: What machine learning courses are available?


19:14:03 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:14:03 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


📊 Hybrid Approach Results:
   Query: "What machine learning courses are available?"
   Token count: 297

   Components:
   - Catalog overview: 259 tokens
   - Specific details (RAG): 25 tokens

📄 Context Structure:
   1. Full catalog overview (all departments)
   2. Detailed info for most relevant courses



### Compare All Three Approaches

Let's test all three with the same query and compare results:

In [14]:
# Test all three approaches
query = "What machine learning courses are available?"

print("=" * 80)
print("COMPARING THREE APPROACHES")
print("=" * 80)

# Approach 1: RAG
messages_rag = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

Available Courses:
{rag_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]
response_rag = llm.invoke(messages_rag)

# Approach 2: Structured View
messages_view = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

{catalog_view}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]
response_view = llm.invoke(messages_view)

# Approach 3: Hybrid
messages_hybrid = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

{hybrid_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]
response_hybrid = llm.invoke(messages_hybrid)

# Display comparison
print(f"""
Query: "{query}"

{'=' * 80}
APPROACH 1: RAG (Hierarchical Search)
{'=' * 80}
Token count: {rag_tokens:,}
Response:
{response_rag.content}

{'=' * 80}
APPROACH 2: Structured View (Pre-Computed)
{'=' * 80}
Token count: {catalog_tokens:,}
Response:
{response_view.content}

{'=' * 80}
APPROACH 3: Hybrid (View + RAG)
{'=' * 80}
Token count: {hybrid_tokens:,}
Response:
{response_hybrid.content}

{'=' * 80}
""")

COMPARING THREE APPROACHES


19:14:05 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


19:14:07 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


19:14:10 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: "What machine learning courses are available?"

APPROACH 1: RAG (Hierarchical Search)
Token count: 25
Response:
Currently, there are no machine learning courses available at Redis University. If you are interested in other topics or need assistance with something else, feel free to ask!

APPROACH 2: Structured View (Pre-Computed)
Token count: 259
Response:
In the Mathematics category, we offer several courses on Linear Algebra for Machine Learning, which are at an intermediate level. Here are the available courses:

- MATH001: Linear Algebra for Machine Learning
- MATH003: Linear Algebra for Machine Learning
- MATH012: Linear Algebra for Machine Learning
- MATH017: Linear Algebra for Machine Learning
- MATH026: Linear Algebra for Machine Learning
- MATH028: Linear Algebra for Machine Learning
- MATH036: Linear Algebra for Machine Learning
- MATH038: Linear Algebra for Machine Learning
- MATH040: Linear Algebra for Machine Learning
- MATH045: Linear Algebra for Machine Learning


### Decision Framework: Which Approach to Use?

Here's how to choose based on YOUR requirements:

| Factor | RAG | Structured Views | Hybrid |
|--------|-----|------------------|--------|
| **Token Efficiency** | ✅ Good (3K) | ✅✅ Excellent (2K) | ⚠️ Moderate (5K) |
| **Response Quality** | ✅ Good (relevant) | ✅ Good (overview) | ✅✅ Excellent (both) |
| **Latency** | ⚠️ Moderate (search) | ✅✅ Fast (cached) | ⚠️ Moderate (search) |
| **Maintenance** | ✅ Low (auto-updates) | ⚠️ Higher (rebuild views) | ⚠️ Higher (both) |
| **Best For** | Specific queries | Overview queries | Real-world systems |

**Decision Process:**

1. **Analyze YOUR data characteristics:**
   - How many items? (10s, 100s, 1000s, millions?)
   - How often does it change? (Real-time, daily, weekly?)
   - What's the average item size? (100 words, 1000 words, 10K words?)

2. **Analyze YOUR query patterns:**
   - Specific queries ("Show me RU101") → RAG
   - Overview queries ("What courses exist?") → Structured Views
   - Mixed queries → Hybrid

3. **Analyze YOUR constraints:**
   - Tight token budget → Structured Views
   - Real-time updates required → RAG
   - Best quality needed → Hybrid

**Example Decision:**

For Redis University:
- ✅ **Data:** 100-500 courses, updated weekly, 200-500 words each
- ✅ **Queries:** Mix of overview ("What's available?") and specific ("ML courses?")
- ✅ **Constraints:** Moderate token budget, weekly updates acceptable
- ✅ **Decision:** **Hybrid approach** (pre-compute catalog + RAG for details)

---

## Part 4: Introduction to Chunking - When and Why

So far, we've worked with course data where each course is a complete, self-contained unit. But what happens when you have **long documents** that contain multiple distinct topics?

This is where **chunking** *may* become necessary—but it's not always the right choice.

### The Critical First Question: Does My Data Need Chunking?

**Chunking is NOT a default step** - it's an engineering decision that depends on multiple factors:

1. **Document type:** Structured records vs. long-form text vs. PDFs
2. **Data characteristics:** Small discrete items vs. large continuous documents
3. **Application requirements:** Query patterns, retrieval precision needs
4. **Embedding model limitations:** Context window size

⚠️ **Important:** With modern long-context models (128K+ tokens), "fitting in context" is less of a concern. The real question is about **retrieval precision** and **data modeling**.

Let's understand when chunking helps—and when it hurts.

### The "Don't Chunk" Strategy: A Valid Option

For structured data and naturally small documents, **not chunking is often the best strategy**:

**Examples where whole-record embedding works better:**
- ✅ **Course catalogs** - Each course is a complete, self-contained unit
- ✅ **Product listings** - All product info should be retrieved together
- ✅ **FAQ entries** - Question + answer form an atomic unit
- ✅ **Database records** - Structured data with natural boundaries
- ✅ **Support tickets** - Single issue with context

**Why "don't chunk" works here:**
- Each unit is **semantically complete** - all relevant info is together
- Natural boundaries exist - chunking would split related information
- Retrieval precision is maximized - you get exactly what's relevant
- Simpler implementation - no chunking logic, no overlap decisions

> 💡 **Key Insight:** For our course catalog, each course is already an optimal retrieval unit. Chunking it would only hurt quality.

In [15]:
# Example: Course data (NO chunking needed)
sample_course_text = transform_course_to_text(all_courses[0])
sample_tokens = count_tokens(sample_course_text)

print(f"""📊 Example: Course Description
{'=' * 80}
{sample_course_text}
{'=' * 80}

Token count: {sample_tokens}
Semantic completeness: ✅ Complete (has all info about this course)
Chunking needed? ❌ NO

Why whole-record embedding works here:
- Self-contained (doesn't reference other sections)
- Semantically complete (has all course details)
- Natural boundary (one course = one retrieval unit)
- Breaking it up would hurt retrieval quality
""")

📊 Example: Course Description
CS008: Natural Language Processing
Department: Computer Science
Credits: 3
Level: advanced
Format: hybrid
Instructor: Cindy Sandoval
Prerequisites: CS001
Description: Processing and understanding human language with machine learning. This course covers text processing, language models, sentiment analysis, and modern NLP techniques. Students will work with transformers, BERT, and GPT models. Projects include building chatbots, text classifiers, and language generation systems.
    

Token count: 93
Semantic completeness: ✅ Complete (has all info about this course)
Chunking needed? ❌ NO

Why whole-record embedding works here:
- Self-contained (doesn't reference other sections)
- Semantically complete (has all course details)
- Natural boundary (one course = one retrieval unit)
- Breaking it up would hurt retrieval quality



### When Chunking May Help

Chunking *can* improve retrieval when documents have certain characteristics:

**Document types that may benefit from chunking:**
- Research papers with multiple distinct sections
- Technical documentation spanning many topics
- Books and long-form content
- Legal contracts with multiple clauses
- Medical records with multiple visits/conditions

**Why chunking helps in these cases:**
- **Multiple distinct topics** - Different sections should be retrieved separately for precision
- **Improves retrieval precision** - Find specific sections, not whole document
- **Better data modeling** - Just like database design, structure affects quality
- **Addresses research-documented context quality problems** (see below)

⚠️ **Note:** Modern embedding models (jina-embeddings-v2, etc.) support 8K+ tokens. Model limits are less of a concern than **retrieval precision**.

### Research Background: Why Long Context Can Hurt

Even with large context windows (128K+ tokens), research shows that **how you structure context matters more than fitting everything in**:

**1. "Lost in the Middle" (Stanford/UC Berkeley, 2023)**

*Source: [arXiv:2307.03172](https://arxiv.org/abs/2307.03172), published in TACL 2024*

LLMs exhibit a **"U-shaped" attention pattern**:
- High recall for information at the **beginning** and **end** of context
- **Significantly degraded performance** for information in the middle
- This happens even in models explicitly designed for long contexts

**Implication for chunking:** For long documents, chunking ensures the relevant section is retrieved and placed prominently in context, rather than buried in the middle of a 50-page document.

**2. "Context Rot" (Chroma Research, 2025)**

*Source: [research.trychroma.com/context-rot](https://research.trychroma.com/context-rot)*

Tested 18 LLMs (GPT-4.1, Claude 4, Gemini 2.5, Qwen3) and found:
- **Performance degrades as input length increases**, even when relevant info is present
- **Distractor effect**: Irrelevant content actively hurts model performance
- Even 4 distractor documents can significantly degrade output quality
- Position of irrelevant content matters (middle is worst)

**Implication for chunking:** Retrieving smaller, focused chunks reduces "distractor tokens" that poison the context.

**3. Needle in the Haystack (NIAH) Benchmark**

*Source: [Greg Kamradt, github.com/gkamradt/LLMTest_NeedleInAHaystack](https://github.com/gkamradt/LLMTest_NeedleInAHaystack)*

Tests whether LLMs can find a specific fact ("needle") buried in long context ("haystack"):
- Models often **fail to retrieve information** even when it's present in context
- Performance varies by position (middle is typically worst)
- **Limitation**: NIAH tests lexical retrieval only, not semantic understanding

**Implication for chunking:** For structured data like course catalogs, NIAH is **irrelevant**—each record IS the needle. Chunking would only fragment complete units.

---

**The Key Insight:**

These research findings don't prescribe a universal chunking rule—they inform your design decisions:

- **Structured records** (courses, products, FAQs): The "lost in the middle" problem typically doesn't apply because each record is already a focused, atomic unit.

- **Long-form documents**: Context rot and positional bias become more relevant as document length increases. Chunking can help surface relevant sections, but over-chunking fragments context.

- **Mixed content types**: Real-world data rarely fits neat categories. Experiment with your actual data and queries.

### Emerging Strategies (2024)

Recent research has introduced new approaches that challenge traditional chunking:

1. **Late Chunking** (Jina AI): Embed the entire document first, then chunk the embeddings. Preserves cross-chunk context without ColBERT-level storage costs.

2. **Contextual Retrieval** (Anthropic): Add LLM-generated context to each chunk before embedding. Can reduce retrieval failures by 49-67%.

3. **Hybrid Search**: Combine vector embeddings with BM25 keyword search. Often outperforms either approach alone.

The key insight: **chunking strategy is a design choice, not a one-size-fits-all prescription**.

In [16]:
# Example: Research paper (NEEDS chunking)
# Let's simulate a long research paper about Redis

research_paper = """
# Optimizing Vector Search Performance in Redis

## Abstract
This paper presents a comprehensive analysis of vector search optimization techniques in Redis,
examining the trade-offs between search quality, latency, and memory usage. We evaluate multiple
indexing strategies including HNSW and FLAT indexes across datasets ranging from 10K to 10M vectors.
Our results demonstrate that careful index configuration can improve search latency by up to 10x
while maintaining 95%+ recall. We also introduce novel compression techniques that reduce memory
usage by 75% with minimal impact on search quality.

## 1. Introduction
Vector databases have become essential infrastructure for modern AI applications, enabling semantic
search, recommendation systems, and retrieval-augmented generation (RAG). Redis, traditionally known
as an in-memory data structure store, has evolved to support high-performance vector search through
the RediSearch module. However, optimizing vector search performance requires understanding complex
trade-offs between multiple dimensions...

[... 5,000 more words covering methodology, experiments, results, discussion ...]

## 2. Background and Related Work
Previous work on vector search optimization has focused primarily on algorithmic improvements to
approximate nearest neighbor (ANN) search. Malkov and Yashunin (2018) introduced HNSW, which has
become the de facto standard for high-dimensional vector search. Johnson et al. (2019) developed
FAISS, demonstrating that product quantization can significantly reduce memory usage...

[... 2,000 more words ...]

## 3. Performance Analysis and Results

### 3.1 HNSW Configuration Trade-offs

Table 1 shows the performance comparison across different HNSW configurations. As M increases from 16 to 64,
we observe significant improvements in recall (0.89 to 0.97) but at the cost of increased latency (2.1ms to 8.7ms)
and memory usage (1.2GB to 3.8GB). The sweet spot for most real-world workloads is M=32 with ef_construction=200,
which achieves 0.94 recall with 4.3ms latency.

Table 1: HNSW Performance Comparison
| M  | ef_construction | Recall@10 | Latency (ms) | Memory (GB) | Build Time (min) |
|----|-----------------|-----------|--------------|-------------|------------------|
| 16 | 100            | 0.89      | 2.1          | 1.2         | 8                |
| 32 | 200            | 0.94      | 4.3          | 2.1         | 15               |
| 64 | 400            | 0.97      | 8.7          | 3.8         | 32               |

The data clearly demonstrates the fundamental trade-off between search quality and resource consumption.
For applications requiring high recall (>0.95), the increased latency and memory costs are unavoidable.

### 3.2 Mathematical Model

The recall-latency trade-off can be modeled as a quadratic function of the HNSW parameters:

Latency(M, ef) = α·M² + β·ef + γ

Where:
- M = number of connections per layer (controls graph connectivity)
- ef = size of dynamic candidate list (controls search breadth)
- α, β, γ = dataset-specific constants (fitted from experimental data)

For our e-commerce dataset, we fitted: α=0.002, β=0.015, γ=1.2 (R²=0.94)

## 4. Implementation Recommendations

Based on our findings, we recommend the following configuration for real-world deployments:

```python
# Optimal HNSW configuration for balanced performance
index_params = {
    "M": 32,                  # Balance recall and latency
    "ef_construction": 200,   # Higher quality index
    "ef_runtime": 100         # Fast search with good recall
}
```

This configuration achieves 0.94 recall with 4.3ms p95 latency, suitable for most real-time applications.

## 5. Discussion and Conclusion
Our findings demonstrate that vector search optimization is fundamentally about understanding
YOUR specific requirements and constraints. There is no one-size-fits-all configuration.
"""

paper_tokens = count_tokens(research_paper)
print(f"Token count: {paper_tokens:,} | Words: ~{len(research_paper.split())}")

Token count: 890 | Words: ~532


**📊 Analysis: Research Paper Example**

**Document:** "Optimizing Vector Search Performance in Redis"

**Structure:** Abstract, Introduction, Background, Methodology, Results, Discussion

**Chunking needed?** ✅ **YES**

**Why This Document May Benefit from Chunking:**

> **Note:** Modern LLMs can handle 128K+ tokens, so "fitting in context" isn't the issue. The real value of chunking is **better data modeling and retrieval precision**.

**1. Retrieval Precision vs. Recall Trade-off**

Without chunking (embed entire paper):
- Query: "What compression techniques were used?"
- Retrieved: Entire 15,000-token paper
- Problem: 80% of retrieved content is irrelevant to the query

With chunking (embed by section):
- Query: "What compression techniques were used?"
- Retrieved: Methodology section (800 tokens)
- Result: 90%+ of retrieved content is directly relevant

**2. Query-Specific Retrieval Patterns**

| Query | Needs | Without Chunking | With Chunking |
|-------|-------|------------------|---------------|
| "What compression techniques?" | Methodology section | Entire paper (15K tokens) | Methodology (800 tokens) |
| "What were recall results?" | Results + Table 1 | Entire paper (15K tokens) | Results section (600 tokens) |
| "How does HNSW work?" | Background + Formula | Entire paper (15K tokens) | Background (500 tokens) |
| "What's the recommended config?" | Discussion + Code | Entire paper (15K tokens) | Discussion (400 tokens) |

**Impact:** 10-20x reduction in irrelevant context, leading to faster responses and better quality.

**💡 Key Insight:** Chunking isn't about fitting in context windows - it's about **data modeling for retrieval**. Just like you wouldn't store all customer data in one database row, you shouldn't embed all document content in one vector.

### Chunking Strategies: Engineering Trade-Offs

Once you've determined that your data needs chunking, the next question is: **How should you chunk it?**

There's no single "best" chunking strategy - the optimal approach depends on YOUR data characteristics and query patterns.

### Strategy 1: Document-Based Chunking (Structure-Aware)

**Concept:** Split documents based on their inherent structure (sections, paragraphs, headings).

**Best for:** Structured documents with clear logical divisions (research papers, technical docs, books).

In [17]:
# Strategy 1: Document-Based Chunking
# Split research paper by sections (using markdown headers)


def chunk_by_structure(text: str, separator: str = "\n## ") -> List[str]:
    """Split text by structural markers (e.g., markdown headers)."""

    # Split by headers
    sections = text.split(separator)

    # Clean and format chunks
    chunks = []
    for i, section in enumerate(sections):
        if section.strip():
            # Add header back (except for first chunk which is title)
            if i > 0:
                chunk = "## " + section
            else:
                chunk = section
            chunks.append(chunk.strip())

    return chunks


# Apply to research paper
structure_chunks = chunk_by_structure(research_paper)

print(f"""📊 Strategy 1: Document-Based (Structure-Aware) Chunking
{'=' * 80}
Original document: {paper_tokens:,} tokens
Number of chunks: {len(structure_chunks)}

Chunk breakdown:
""")

for i, chunk in enumerate(structure_chunks):
    chunk_tokens = count_tokens(chunk)
    # Show first 100 chars of each chunk
    preview = chunk[:300].replace("\n", " ")
    print(f"   Chunk {i+1}: {chunk_tokens:,} tokens - {preview}...\n")

📊 Strategy 1: Document-Based (Structure-Aware) Chunking
Original document: 890 tokens
Number of chunks: 7

Chunk breakdown:

   Chunk 1: 8 tokens - # Optimizing Vector Search Performance in Redis...

   Chunk 2: 108 tokens - ## Abstract This paper presents a comprehensive analysis of vector search optimization techniques in Redis, examining the trade-offs between search quality, latency, and memory usage. We evaluate multiple indexing strategies including HNSW and FLAT indexes across datasets ranging from 10K to 10M vec...

   Chunk 3: 98 tokens - ## 1. Introduction Vector databases have become essential infrastructure for modern AI applications, enabling semantic search, recommendation systems, and retrieval-augmented generation (RAG). Redis, traditionally known as an in-memory data structure store, has evolved to support high-performance ve...

   Chunk 4: 98 tokens - ## 2. Background and Related Work Previous work on vector search optimization has focused primarily on algorithmic im

**Strategy 1 Analysis:**

✅ **Advantages:**
- Respects document structure (sections stay together)
- Semantically coherent (each chunk is a complete section)
- Easy to implement for structured documents
- **Keeps tables, formulas, and code WITH their context**

⚠️ **Trade-offs:**
- Variable chunk sizes (some sections longer than others)
- Requires documents to have clear structure
- May create chunks that are still too large

🎯 **Best for:**
- Research papers with clear sections
- Technical documentation with headers
- Books with chapters/sections

### Strategy 2: Fixed-Size Chunking (Token-Based)

**Concept:** Split text into chunks of a predetermined size (e.g., 512 tokens) with overlap.

**Best for:** Unstructured text, quick prototyping, when you need consistent chunk sizes.

In [18]:
# Strategy 2: Fixed-Size Chunking (Using LangChain)
# Industry-standard approach with smart boundary detection

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create text splitter with smart boundary detection
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Target chunk size in characters
    chunk_overlap=100,  # Overlap to preserve context
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""],  # Try these in order
    is_separator_regex=False,
)

print("🔄 Running fixed-size chunking with LangChain...")
print("   Trying to split on: paragraphs → sentences → words → characters\n")

# Apply to research paper
fixed_chunks_docs = text_splitter.create_documents([research_paper])
fixed_chunks = [doc.page_content for doc in fixed_chunks_docs]

print(f"""📊 Strategy 2: Fixed-Size (LangChain) Chunking
{'=' * 80}
Original document: {paper_tokens:,} tokens
Target chunk size: 800 characters (~200 words)
Overlap: 100 characters
Number of chunks: {len(fixed_chunks)}

Chunk breakdown:
""")

for i, chunk in enumerate(fixed_chunks[:5]):  # Show first 5
    chunk_tokens = count_tokens(chunk)
    preview = chunk[:100].replace("\n", " ")
    print(f"   Chunk {i+1}: {chunk_tokens:,} tokens - {preview}...")

print(f"... ({len(fixed_chunks) - 5} more chunks)")

🔄 Running fixed-size chunking with LangChain...
   Trying to split on: paragraphs → sentences → words → characters

📊 Strategy 2: Fixed-Size (LangChain) Chunking
Original document: 890 tokens
Target chunk size: 800 characters (~200 words)
Overlap: 100 characters
Number of chunks: 7

Chunk breakdown:

   Chunk 1: 117 tokens - # Optimizing Vector Search Performance in Redis  ## Abstract This paper presents a comprehensive ana...
   Chunk 2: 98 tokens - ## 1. Introduction Vector databases have become essential infrastructure for modern AI applications,...
   Chunk 3: 134 tokens - [... 5,000 more words covering methodology, experiments, results, discussion ...]  ## 2. Background ...
   Chunk 4: 129 tokens - ## 3. Performance Analysis and Results  ### 3.1 HNSW Configuration Trade-offs  Table 1 shows the per...
   Chunk 5: 206 tokens - Table 1: HNSW Performance Comparison | M  | ef_construction | Recall@10 | Latency (ms) | Memory (GB)...
... (2 more chunks)


**Strategy 2 Analysis:**

✅ **Advantages:**
- **Respects natural boundaries**: Tries paragraphs → sentences → words → characters
- Consistent chunk sizes (predictable token usage)
- Works on any text (structured or unstructured)
- **Doesn't split mid-sentence** (unless absolutely necessary)

⚠️ **Trade-offs:**
- Ignores document structure (doesn't understand sections)
- Can break semantic coherence (may split related content)
- Overlap creates redundancy (increases storage/cost)

🎯 **Best for:**
- Unstructured text (no clear sections)
- Quick prototyping and baselines
- When consistent chunk sizes are required

### Strategy 3: Semantic Chunking (Meaning-Based)

**Concept:** Split text based on semantic similarity using embeddings - create new chunks when topic changes significantly.

**How it works:**
1. Split text into sentences or paragraphs
2. Generate embeddings for each segment
3. Calculate similarity between consecutive segments
4. Create chunk boundaries where similarity drops (topic shift detected)

**Best for:** Dense academic text, legal documents, narratives where semantic boundaries don't align with structure.

In [19]:
# Strategy 3: Semantic Chunking (Using LangChain)
# Industry-standard approach with local embeddings (no API costs!)

from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
import os

# Suppress tokenizer warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize local embeddings (no API costs!)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# Create semantic chunker with percentile-based breakpoint detection
semantic_chunker = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="percentile",  # Split at bottom 25% of similarities
    breakpoint_threshold_amount=25,  # 25th percentile
    buffer_size=1,  # Compare consecutive sentences
)

print("🔄 Running semantic chunking with LangChain...")
print("   Using local embeddings (sentence-transformers/all-MiniLM-L6-v2)")
print("   Breakpoint detection: 25th percentile of similarity scores\n")

# Apply to research paper
semantic_chunks_docs = semantic_chunker.create_documents([research_paper])

# Extract text from Document objects
semantic_chunks = [doc.page_content for doc in semantic_chunks_docs]

print(f"""📊 Strategy 3: Semantic (LangChain) Chunking
{'=' * 80}
Original document: {paper_tokens:,} tokens
Number of chunks: {len(semantic_chunks)}

Chunk breakdown:
""")

for i, chunk in enumerate(semantic_chunks[:5]):  # Show first 5
    chunk_tokens = count_tokens(chunk)
    preview = chunk[:100].replace("\n", " ")
    print(f"   Chunk {i+1}: {chunk_tokens:,} tokens - {preview}...")

if len(semantic_chunks) > 5:
    print(f"... ({len(semantic_chunks) - 5} more chunks)")

19:14:27 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


🔄 Running semantic chunking with LangChain...
   Using local embeddings (sentence-transformers/all-MiniLM-L6-v2)
   Breakpoint detection: 25th percentile of similarity scores



📊 Strategy 3: Semantic (LangChain) Chunking
Original document: 890 tokens
Number of chunks: 20

Chunk breakdown:

   Chunk 1: 70 tokens -  # Optimizing Vector Search Performance in Redis  ## Abstract This paper presents a comprehensive an...
   Chunk 2: 26 tokens - Our results demonstrate that careful index configuration can improve search latency by up to 10x whi...
   Chunk 3: 22 tokens - We also introduce novel compression techniques that reduce memory usage by 75% with minimal impact o...
   Chunk 4: 4 tokens - ## 1....
   Chunk 5: 60 tokens - Introduction Vector databases have become essential infrastructure for modern AI applications, enabl...
... (15 more chunks)


**Strategy 3 Analysis:**

✅ **Advantages:**
- **Meaning-aware**: Chunks based on topic shifts, not arbitrary boundaries
- **Adaptive**: Chunk sizes vary based on content coherence
- **Better retrieval**: Each chunk is semantically focused
- **Free**: Uses local embeddings (no API costs)

⚠️ **Trade-offs:**
- Slower processing (requires embedding generation)
- Variable chunk sizes (harder to predict token usage)
- May not respect document structure (sections, headers)
- Requires tuning (threshold, buffer size)

🎯 **Best for:**
- Dense academic text
- Legal documents
- Narratives and stories
- Content where semantic boundaries don't align with structure

### Comparing Chunking Strategies: Decision Framework

Now let's compare all strategies side-by-side:

In [20]:
print(f"""
{'=' * 80}
CHUNKING STRATEGY COMPARISON
{'=' * 80}

Document: Research Paper ({paper_tokens:,} tokens)

Strategy              | Chunks | Avg Size | Complexity | Best For
--------------------- | ------ | -------- | ---------- | --------
Document-Based        | {len(structure_chunks):>6} | {sum(count_tokens(c) for c in structure_chunks) // len(structure_chunks):>8} | Low        | Structured docs
Fixed-Size            | {len(fixed_chunks):>6} | {sum(count_tokens(c) for c in fixed_chunks) // len(fixed_chunks):>8} | Low        | Unstructured text
Semantic              | {len(semantic_chunks):>6} | {sum(count_tokens(c) for c in semantic_chunks) // len(semantic_chunks):>8} | High       | Dense academic text

{'=' * 80}
""")


CHUNKING STRATEGY COMPARISON

Document: Research Paper (890 tokens)

Strategy              | Chunks | Avg Size | Complexity | Best For
--------------------- | ------ | -------- | ---------- | --------
Document-Based        |      7 |      127 | Low        | Structured docs
Fixed-Size            |      7 |      137 | Low        | Unstructured text
Semantic              |     20 |       44 | High       | Dense academic text




### YOUR Chunking Decision Framework

Chunking strategy is a **design choice** that depends on your specific context. There's no universal "correct" chunk size.

**Step 1: Start with Document Type**

| Document Type | Default Approach | Reasoning |
|---------------|------------------|----------|
| **Structured records** (courses, products, FAQs) | Don't chunk | Natural boundaries already exist |
| **Long-form text** (papers, books, docs) | Consider chunking | May need retrieval precision |
| **PDFs with visual layout** | Page-level | Preserves tables, figures |
| **Code** | Function/class boundaries | Semantic structure matters |

**Step 2: Evaluate These Factors**

1. **Semantic completeness:** Is each item self-contained?
   - ✅ Yes → Don't chunk (preserve natural boundaries)
   - ❌ No → Consider chunking strategy

2. **Query patterns:** What will users ask?
   - Specific facts → Smaller, focused chunks help
   - Summaries/overviews → Larger chunks or hierarchical
   - Mixed → Consider hierarchical approach

3. **Topic density:** How many distinct topics per document?
   - Single topic → Whole-document embedding often works
   - Multiple distinct topics → Chunking may improve precision

**Example Decisions:**

| Domain | Data Characteristics | Decision | Why |
|--------|---------------------|----------|-----|
| **Course Catalog** | Small, self-contained records | **Don't chunk** | Each course is a complete retrieval unit |
| **Research Papers** | Multi-section, dense topics | Document-Based | Sections are natural semantic units |
| **Support Tickets** | Single issue per ticket | **Don't chunk** | Already at optimal granularity |
| **Legal Contracts** | Nested structure, many clauses | Hierarchical | Need both overview and clause-level detail |

> 💡 **Key Takeaway:** Ask "What is my natural retrieval unit?" before deciding on a chunking strategy. For many structured data use cases, the answer is "don't chunk."

---

## Part 5: Building Practical Context Pipelines

Now that you understand data transformation and chunking, let's discuss how to build reusable pipelines.

### Three Pipeline Architectures

There are three main approaches to context preparation in real-world applications:

### Architecture 1: Request-Time Processing

**Concept:** Transform data on-the-fly when a query arrives.

```
User Query → Retrieve Raw Data → Transform → Chunk (if needed) → Embed → Search → Return Context
```

**Pros:**
- ✅ Always up-to-date (no stale data)
- ✅ No pre-processing required
- ✅ Simple to implement

**Cons:**
- ❌ Higher latency (processing happens during request)
- ❌ Repeated work (same transformations for every query)
- ❌ Not suitable for large datasets

**Best for:**
- Small datasets (< 1,000 documents)
- Frequently changing data
- Simple transformations

### Architecture 2: Batch Processing

**Concept:** Pre-process all data in batches (nightly, weekly) and store results.

```
[Scheduled Job]
Raw Data → Extract → Clean → Transform → Chunk → Embed → Store in Vector DB

[Query Time]
User Query → Search Vector DB → Return Pre-Processed Context
```

**Pros:**
- ✅ Fast query time (all processing done ahead)
- ✅ Efficient (process once, use many times)
- ✅ Can use expensive transformations (LLM-based chunking, semantic analysis)

**Cons:**
- ❌ Data can be stale (until next batch run)
- ❌ Requires scheduling infrastructure
- ❌ Higher storage costs (store processed data)

**Best for:**
- Large datasets (> 10,000 documents)
- Infrequently changing data (daily/weekly updates)
- Complex transformations (semantic chunking, LLM summaries)

### Architecture 3: Event-Driven Processing

**Concept:** Process data as it changes (real-time updates).

```
Data Change Event → Trigger Pipeline → Extract → Clean → Transform → Chunk → Embed → Update Vector DB

[Query Time]
User Query → Search Vector DB → Return Context
```

**Pros:**
- ✅ Always up-to-date (real-time)
- ✅ Fast query time (pre-processed)
- ✅ Efficient (only process changed data)

**Cons:**
- ❌ Complex infrastructure (event streams, queues)
- ❌ Requires change detection
- ❌ Higher operational complexity

**Best for:**
- Real-time data (news, social media, live updates)
- Large datasets that change frequently
- When both freshness and speed are critical

### Choosing YOUR Pipeline Architecture

Use this decision tree:

**Question 1: How often does your data change?**
- Real-time (seconds/minutes) → Event-Driven
- Frequently (hourly/daily) → Batch or Event-Driven
- Infrequently (weekly/monthly) → Batch
- Rarely (manual updates) → Request-Time or Batch

**Question 2: How large is your dataset?**
- Small (< 1,000 docs) → Request-Time
- Medium (1,000-100,000 docs) → Batch
- Large (> 100,000 docs) → Batch or Event-Driven

**Question 3: What are your latency requirements?**
- Real-time (< 100ms) → Batch or Event-Driven (pre-processed)
- Interactive (< 1s) → Any approach
- Batch queries → Request-Time acceptable

**Example Decision:**

For Redis University:
- ✅ **Data changes:** Weekly (new courses added)
- ✅ **Dataset size:** 100-500 courses (medium)
- ✅ **Latency:** Interactive (< 1s acceptable)
- ✅ **Transformations:** Moderate (structured views + embeddings)
- ✅ **Decision:** **Batch Processing** (weekly job to rebuild catalog + embeddings)

### Example: Batch Processing Pipeline for Redis University

In [21]:
# Example: Batch Processing Pipeline
# This would run as a scheduled job (e.g., weekly)


async def batch_process_courses():
    """
    Batch processing pipeline for Redis University courses.
    Runs weekly to update catalog and embeddings.
    """

    print("=" * 80)
    print("BATCH PROCESSING PIPELINE - Redis University Courses")
    print("=" * 80)

    # Step 1: Extract
    print("\n[Step 1/5] Extracting course data...")
    all_courses_batch = await course_manager.get_all_courses()
    print(f"   ✅ Extracted {len(all_courses_batch)} courses")

    # Show sample raw data
    if all_courses_batch:
        sample = all_courses_batch[0]
        print(f"\n   📄 Sample raw course:")
        print(f"      {sample.course_code}: {sample.title}")
        print(f"      Department: {sample.department}, Credits: {sample.credits}, Level: {sample.difficulty_level.value}")

    # Step 2: Clean
    print("\n[Step 2/5] Cleaning data...")
    # Remove test courses, validate fields, etc.
    cleaned_courses = [
        c for c in all_courses_batch if c.course_code.startswith(("RU", "CS", "MATH"))
    ]
    print(f"   ✅ Cleaned: {len(cleaned_courses)} courses (removed {len(all_courses_batch) - len(cleaned_courses)} test courses)")

    # Step 3: Transform
    print("\n[Step 3/5] Transforming to LLM-friendly format...")
    transformed_courses = [transform_course_to_text(c) for c in cleaned_courses]
    total_tokens = sum(count_tokens(t) for t in transformed_courses)
    print(f"   ✅ Transformed: {len(transformed_courses)} courses ({total_tokens:,} total tokens)")

    # Step 4: Create Structured Views
    print("\n[Step 4/5] Creating structured catalog view...")
    catalog_view_batch = await create_catalog_view()
    catalog_tokens_batch = count_tokens(catalog_view_batch)
    redis_client.set("course_catalog_view", catalog_view_batch)
    redis_client.set("course_catalog_view:updated", "2024-09-30")
    print(f"   ✅ Created catalog view ({catalog_tokens_batch:,} tokens)")
    print(f"   ✅ Cached in Redis")

    # Step 5: Store (in production, would also create embeddings and store in vector DB)
    print("\n[Step 5/5] Storing processed data...")
    for i, (course, text) in enumerate(zip(cleaned_courses, transformed_courses)):
        key = f"course:processed:{course.course_code}"
        redis_client.set(key, text)
    print(f"   ✅ Stored {len(cleaned_courses)} processed courses in Redis")

    print("\n" + "=" * 80)
    print("BATCH PROCESSING COMPLETE")
    print("=" * 80)
    print(f"""
Summary:
- Courses processed: {len(cleaned_courses)}
- Total tokens: {total_tokens:,}
- Catalog view tokens: {catalog_tokens_batch:,}
- Storage: Redis
- Next run: 2024-10-07 (weekly)
""")


# Run the batch pipeline
await batch_process_courses()

BATCH PROCESSING PIPELINE - Redis University Courses

[Step 1/5] Extracting course data...


19:14:30 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


   ✅ Extracted 50 courses

   📄 Sample raw course:
      CS008: Natural Language Processing
      Department: Computer Science, Credits: 3, Level: advanced

[Step 2/5] Cleaning data...
   ✅ Cleaned: 50 courses (removed 0 test courses)

[Step 3/5] Transforming to LLM-friendly format...
   ✅ Transformed: 50 courses (4,765 total tokens)

[Step 4/5] Creating structured catalog view...
   ✅ Created catalog view (259 tokens)
   ✅ Cached in Redis

[Step 5/5] Storing processed data...
   ✅ Stored 50 processed courses in Redis

BATCH PROCESSING COMPLETE

Summary:
- Courses processed: 50
- Total tokens: 4,765
- Catalog view tokens: 259
- Storage: Redis
- Next run: 2024-10-07 (weekly)



---

## Part 6: Quality Optimization - Measuring and Improving Context

### The Systematic Optimization Process

Now that you understand data engineering and context pipelines, let's learn how to systematically optimize context quality.

**The Process:**
```
1. Define Quality Metrics (domain-specific)
   ↓
2. Establish Baseline (measure current performance)
   ↓
3. Experiment (try different approaches)
   ↓
4. Measure (compare against metrics)
   ↓
5. Iterate (refine based on results)
```

### Step 1: Define Quality Metrics for YOUR Domain

**The Problem with Generic Metrics:**

Don't aim for "95% accuracy on benchmark X" - that benchmark wasn't designed for YOUR domain.

**DO this instead:** Define what "quality" means for YOUR domain, then measure it.

### The Four Quality Dimensions

Every context engineering solution should be evaluated across four dimensions:

1. **Relevance** - Does context include information needed to answer the query?
2. **Completeness** - Does context include ALL necessary information?
3. **Efficiency** - Is context optimized for token usage?
4. **Accuracy** - Is context factually correct and up-to-date?

Different domains prioritize these differently.

In [22]:
# Define domain-specific quality metrics

quality_metrics = {
    "Relevance": {
        "definition": "Does context include courses relevant to the user's query?",
        "metric": "% of queries where retrieved courses match query intent",
        "measurement": "Manual review of 50 sample queries",
        "target": ">90%",
        "why_important": "Irrelevant courses waste tokens and confuse users",
    },
    "Completeness": {
        "definition": "Does context include all information needed to answer?",
        "metric": "% of responses that mention all prerequisites when asked",
        "measurement": "Automated check: parse response for prerequisite mentions",
        "target": "100%",
        "why_important": "Missing prerequisites leads to hallucinations",
    },
    "Efficiency": {
        "definition": "Is context optimized for token usage?",
        "metric": "Average tokens per query",
        "measurement": "Token counter on all context strings",
        "target": "<5,000 tokens",
        "why_important": "Exceeding budget increases cost and latency",
    },
    "Accuracy": {
        "definition": "Is context factually correct and up-to-date?",
        "metric": "% of responses with correct course information",
        "measurement": "Manual review against course database",
        "target": ">95%",
        "why_important": "Incorrect information damages trust",
    },
}

print("""QUALITY METRICS FOR REDIS UNIVERSITY COURSE ADVISOR
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━""")
for dimension, details in quality_metrics.items():
    print(f"""
{dimension}:
  Definition: {details['definition']}
  Metric: {details['metric']}
  How to measure: {details['measurement']}
  Target: {details['target']}
  Why important: {details['why_important']}""")
print("\n" + "━" * 80)

QUALITY METRICS FOR REDIS UNIVERSITY COURSE ADVISOR
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Relevance:
  Definition: Does context include courses relevant to the user's query?
  Metric: % of queries where retrieved courses match query intent
  How to measure: Manual review of 50 sample queries
  Target: >90%
  Why important: Irrelevant courses waste tokens and confuse users

Completeness:
  Definition: Does context include all information needed to answer?
  Metric: % of responses that mention all prerequisites when asked
  How to measure: Automated check: parse response for prerequisite mentions
  Target: 100%
  Why important: Missing prerequisites leads to hallucinations

Efficiency:
  Definition: Is context optimized for token usage?
  Metric: Average tokens per query
  How to measure: Token counter on all context strings
  Target: <5,000 tokens
  Why important: Exceeding budget increases cost and latency

Accuracy:
  Definition: Is context fact

### Key Insight: Metrics Must Be Domain-Specific

Notice how these metrics are specific to the course advisor domain:

**Relevance metric:**
- ❌ Generic: "Cosine similarity > 0.8"
- ✅ Domain-specific: "Retrieved courses match query intent"

**Completeness metric:**
- ❌ Generic: "Context includes top-5 search results"
- ✅ Domain-specific: "All prerequisites mentioned when asked"

**Efficiency metric:**
- ❌ Generic: "Minimize tokens"
- ✅ Domain-specific: "<5,000 tokens (fits our budget)"

**Accuracy metric:**
- ❌ Generic: "95% on MMLU benchmark"
- ✅ Domain-specific: "Correct course information vs. database"

**Your metrics should reflect YOUR domain's requirements, not generic benchmarks.**

### Key Takeaways: Quality Optimization

1. **Define Domain-Specific Metrics** - Don't rely on generic benchmarks
2. **Measure Systematically** - Baseline → Experiment → Measure → Iterate
3. **Balance Trade-offs** - Relevance vs. Efficiency, Completeness vs. Token Budget
4. **Test Before Deployment** - Validate with real queries from your domain
5. **Iterate Continuously** - Quality optimization is ongoing, not one-time

**The Engineering Mindset:**
- Context quality is measurable
- Optimization is systematic, not guesswork
- Domain-specific metrics matter more than generic benchmarks
- Testing and iteration are essential

---

## Summary and Key Takeaways

### What You Learned

**1. Context is Data - and Data Requires Engineering**
- Context isn't just "data you feed to an LLM"
- It requires systematic transformation: Raw → Clean → Transform → Optimize → Store
- Engineering discipline: requirements analysis, design decisions, quality metrics, testing

**2. The Data Engineering Pipeline**
- Extract: Get raw data from sources
- Clean: Remove noise, fix inconsistencies
- Transform: Structure for LLM consumption
- Optimize: Reduce tokens, improve clarity
- Store: Choose storage strategy (RAG, Views, Hybrid)

**3. Three Engineering Approaches**
- **RAG:** Semantic search for relevant data (good for specific queries)
- **Structured Views:** Pre-computed summaries (excellent for overviews)
- **Hybrid:** Combine both (best for real-world use)

**4. Chunking is a Design Choice, Not a Default**
- **"Don't chunk"** is a valid strategy for structured data (courses, products, FAQs)
- Consider chunking when documents have multiple distinct topics or need retrieval precision
- Three strategies: Document-Based, Fixed-Size, Semantic
- Choose based on YOUR data type, query patterns, and application requirements

**5. Context Pipeline Architectures**
- **Request-Time:** Process on-the-fly (simple, always fresh, higher latency)
- **Batch:** Pre-process in batches (fast queries, can be stale)
- **Event-Driven:** Process on changes (real-time, complex infrastructure)

**6. Quality Optimization**
- Define domain-specific quality metrics
- Systematic optimization: Baseline → Experiment → Measure → Iterate
- Four dimensions: Relevance, Completeness, Efficiency, Accuracy

### The Engineering Mindset

Every decision should be based on **YOUR specific requirements:**

1. **Analyze YOUR data:** Size, structure, update frequency, topic density
2. **Analyze YOUR queries:** Specific vs. overview, single vs. cross-section
3. **Analyze YOUR constraints:** Token budget, latency, quality requirements
4. **Make informed decisions:** Choose approaches that match YOUR needs
5. **Measure and iterate:** Test with real queries, measure quality, optimize

**Remember:** There is no "best practice" that works for everyone. Context engineering is about making deliberate, informed choices based on YOUR domain, application, and constraints.

---

## Practice Exercises

### Exercise 1: Analyze Your Data
Think about a dataset you work with. Answer these questions:
1. What is the natural retrieval unit? (record, document, section?)
2. Does it need chunking? Why or why not?
3. Which engineering approach would you use? (RAG, Structured Views, Hybrid)

### Exercise 2: Design Quality Metrics
For your domain, define:
1. What does "relevance" mean?
2. What does "completeness" mean?
3. What is your token budget?
4. How will you measure accuracy?

### Exercise 3: Choose a Pipeline Architecture
Based on your data characteristics:
1. How often does your data change?
2. How large is your dataset?
3. What are your latency requirements?
4. Which architecture would you choose? (Request-Time, Batch, Event-Driven)

---

## What's Next?

### Module 4: Memory Systems for Context Engineering

Now that you can engineer high-quality retrieved context, you'll learn to manage conversation context:
- **Working Memory:** Track conversation history within a session
- **Long-term Memory:** Remember user preferences across sessions
- **Memory-Enhanced RAG:** Combine all four context types
- **Redis Agent Memory Server:** Automatic memory extraction and retrieval

```
Module 1: Context Engineering Fundamentals
    ↓
Module 2: RAG Fundamentals ← Completed
    ↓
Module 3: Data Engineering for Context ← You are here
    ↓
Module 4: Memory Systems ← Next
    ↓
Module 5: Building Agents (Complete System)
```

---

## Additional Resources

**Chunking Strategies:**
- [LangChain Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)
- [LlamaIndex Node Parsers](https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/)

**Data Engineering for LLMs:**
- [OpenAI Best Practices](https://platform.openai.com/docs/guides/prompt-engineering)
- [Anthropic Prompt Engineering](https://docs.anthropic.com/claude/docs/prompt-engineering)

**Vector Databases:**
- [Redis Vector Search Documentation](https://redis.io/docs/stack/search/reference/vectors/)
- [RedisVL Python Library](https://github.com/RedisVentures/redisvl)

